In [1]:
import pandas as pd

df = pd.read_csv(r'data\weather_data_nan.csv')

df

,day,temperature,windspeed,event
0,1/1/2017,32.0,6.0,Rain
1,1/4/2017,NaN,9.0,Sunny
2,1/5/2017,28.0,NaN,Snow
3,1/6/2017,NaN,7.0,NaN
4,1/7/2017,32.0,NaN,Rain
5,1/8/2017,NaN,NaN,Sunny
6,1/9/2017,NaN,NaN,NaN
7,1/10/2017,34.0,8.0,Cloudy
8,1/11/2017,40.0,12.0,Sunny


# Parsing dates
We can see that the dates are strings

In [2]:
type(df['day'][0])

str

We can parse the days using the `parse_dates` kwarg

In [3]:
df = pd.read_csv(r'data\weather_data_nan.csv', parse_dates=["day"])
df.set_index('day', inplace=True)
df

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-09,NaN,NaN,NaN
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


In [4]:
type(df.index[0])

pandas._libs.tslibs.timestamps.Timestamp

We can also specify the method of parsing dates by passing in a function to the `date_parser` kwarg.

In [5]:
df = pd.read_csv(r'data\weather_data_nan.csv', parse_dates=["day"], date_parser=pd.to_datetime)
df.set_index('day', inplace=True)
df

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-09,NaN,NaN,NaN
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


# Filling nan values

We can fill in missing values for the whole DataFrame or specific columns using the `fillna` method.

In [6]:
df.fillna(0)

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,0.0,9.0,Sunny
2017-01-05,28.0,0.0,Snow
2017-01-06,0.0,7.0,0
2017-01-07,32.0,0.0,Rain
2017-01-08,0.0,0.0,Sunny
2017-01-09,0.0,0.0,0
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


`fillna` does not replace the original DataFrame, but returns a new one

In [7]:
df

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-09,NaN,NaN,NaN
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


Before, we set all the columns to 0, but that doesn't always make sense. For example, a 0 in the `event` column doesn't make any sense.

We can specify the `fillna` value for each column, by passing in a dictionary.

In [8]:
df.fillna({
    'temperature': 0,
    'windspeed': 0.0,
    'event': 'No event'
})

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,0.0,9.0,Sunny
2017-01-05,28.0,0.0,Snow
2017-01-06,0.0,7.0,No event
2017-01-07,32.0,0.0,Rain
2017-01-08,0.0,0.0,Sunny
2017-01-09,0.0,0.0,No event
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


However, assiging 0 to numerical columns may also cause errors. For example, for the days that temperature is 0 means it is freezing, yet event is sunny.

We can carry forward value of the previous column by specifying the `method` as `ffill` (forward fill).

In [9]:
df.fillna(method='ffill')

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,32.0,9.0,Sunny
2017-01-05,28.0,9.0,Snow
2017-01-06,28.0,7.0,Snow
2017-01-07,32.0,7.0,Rain
2017-01-08,32.0,7.0,Sunny
2017-01-09,32.0,7.0,Sunny
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


We can also carry backward the value from the next cell if a cell is NaN by using `bfill`.

Sometimes, carrying forward one value to too many rows may cause our original dataset to change too much. So we might impose a `limit` to restrict the number of times we carry forward or backward.

In [10]:
df.fillna(method='ffill', limit=1)

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,32.0,9.0,Sunny
2017-01-05,28.0,9.0,Snow
2017-01-06,28.0,7.0,Snow
2017-01-07,32.0,7.0,Rain
2017-01-08,32.0,NaN,Sunny
2017-01-09,NaN,NaN,Sunny
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


# Interpolation to fill nan values

Using __linear interpolation__ to fill __numerical values__

In [11]:
df.interpolate()

,temperature,windspeed,event
day,,,
2017-01-01,32.000000,6.00,Rain
2017-01-04,30.000000,9.00,Sunny
2017-01-05,28.000000,8.00,Snow
2017-01-06,30.000000,7.00,NaN
2017-01-07,32.000000,7.25,Rain
2017-01-08,32.666667,7.50,Sunny
2017-01-09,33.333333,7.75,NaN
2017-01-10,34.000000,8.00,Cloudy
2017-01-11,40.000000,12.00,Sunny


We also use other [Interpolation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html) technieuqs such as cubic, quadratic, polynomial etc by using the `method` kwarg.

Using __linear interpolation__ can sometimes lead to some inaccuracies in data. 
- For example our second entry was interpolated as 30, right between the previous 32 and the next 28.

- However, if we see the __day__ we can see that it is closer to the third entry when the temperature was 28

For this we can set the `method` to `time`, to interpolate the 'temperature' column values by 'time'

In [12]:
df.interpolate(method='time')

,temperature,windspeed,event
day,,,
2017-01-01,32.000000,6.00,Rain
2017-01-04,29.000000,9.00,Sunny
2017-01-05,28.000000,8.00,Snow
2017-01-06,30.000000,7.00,NaN
2017-01-07,32.000000,7.25,Rain
2017-01-08,32.666667,7.50,Sunny
2017-01-09,33.333333,7.75,NaN
2017-01-10,34.000000,8.00,Cloudy
2017-01-11,40.000000,12.00,Sunny


# Droping rows with missing values

Sometimes, it may be more useful for us to simply drop the rows with missing values. We can do this with the DataFrame's `dropna` method.

In [13]:
df

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-09,NaN,NaN,NaN
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


In [14]:
df.dropna()

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


But, maybe this way we are droppig too many of the rows. Maybe we want to drop a row ___if all the columns are empty___.

We can do this if we specify `'all'` in the `how` kwarg.

In [15]:
df.dropna(how='all')

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


If we want to keep a row if it has ___atleast 1 non-na value___, then we can use the number using the `thresh` (threshold) kwarg.

In [16]:
df.dropna(thresh=1)

,temperature,windspeed,event
day,,,
2017-01-01,32.0,6.0,Rain
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-10,34.0,8.0,Cloudy
2017-01-11,40.0,12.0,Sunny


# Inserting missing index
Some of our ___day___(s) are missing. If we want to fill them in we must make a new `Datetimeindex` and reassign our DataFrame's index.

In [17]:
# Making a Pandas date_range object
dt_range = pd.date_range("01-01-2017", "01-11-2017")
# Converting it into a Datetimeindex
new_index = pd.DatetimeIndex(dt_range)
# Resetting our index
df.reindex(new_index)

,temperature,windspeed,event
2017-01-01,32.0,6.0,Rain
2017-01-02,NaN,NaN,NaN
2017-01-03,NaN,NaN,NaN
2017-01-04,NaN,9.0,Sunny
2017-01-05,28.0,NaN,Snow
2017-01-06,NaN,7.0,NaN
2017-01-07,32.0,NaN,Rain
2017-01-08,NaN,NaN,Sunny
2017-01-09,NaN,NaN,NaN
2017-01-10,34.0,8.0,Cloudy


##### Note:
___Remember!__ these methods all return new DataFrames. They do not change the DataFrame `inplace`. Change the original DataFrame either by setting the kwarg `inplace = True` or setting `df = returned_df`